# Search and query RxRx1, RxRx2

In [1]:
!lamin load sunnyosun/rxrx1-2

2023-09-21 14:30:30,770:INFO - Found credentials in shared credentials file: ~/.aws/credentials
❗ updating local SQLite & locking cloud SQLite (sync back & unlock: lamin close)
💡 loaded instance: sunnyosun/rxrx1-2


In [2]:
import lamindb as ln
import lnschema_bionty as lb
import lnschema_lamin1 as ln1

2023-09-21 14:30:37,954:INFO - Found credentials in shared credentials file: ~/.aws/credentials


💡 loaded instance: sunnyosun/rxrx1-2 (lamindb 0.54.0)


In [3]:
ln.track()

💡 notebook imports: duckdb==0.8.1 lamindb==0.54.0 lnschema_bionty==0.31.1 lnschema_lamin1==0.23.0
💡 Transform(id='sx3wFSwnhCYYz8', name='Search and query RxRx1, RxRx2', short_name='query-rxrx', version='0', type='notebook', updated_at=2023-09-21 12:30:46, created_by_id='kmvZDIX9')
💡 Run(id='lyIGDhFyutVCERkiriqp', run_at=2023-09-21 12:30:46, transform_id='sx3wFSwnhCYYz8', created_by_id='kmvZDIX9')
💡   parent transforms:
   - Transform(id='Zo0qJt4IQPsbz8', name='Validate and register RxRx1 metadata', short_name='rxrx1-register', version='0', type='notebook', updated_at=2023-09-21 12:06:11, created_by_id='kmvZDIX9')
   - Transform(id='kq1P1Aho94siz8', name='Register RxRx1 metadata and embedding files', short_name='rxrx1-download', version='0', type='notebook', updated_at=2023-09-19 10:57:46, created_by_id='kmvZDIX9')


In [3]:
features = ln.Feature.lookup(return_field="name")
cell_lines = lb.CellLine.lookup(return_field="abbr")
sirnas = ln1.Treatment.lookup(return_field="name")
wells = ln1.Well.lookup(return_field="name")

## Pandas

In [4]:
file = ln.File.filter(key="rxrx1/metadata.parquet").one()
df = file.load()

In [5]:
df[
    (df.cell_type == cell_lines.hep_g2_cell)
    & (df.sirna == sirnas.s19486)
    & (df.well == wells.l20)
    & (df.plate == "3")
    & (df.site == "2")
]

,site_id,well_id,cell_type,dataset,experiment,plate,well,site,well_type,sirna,sirna_id,file_keys
9097,HEPG2-11_3_L20_2,HEPG2-11_3_L20,HEPG2,test,HEPG2-11,3,L20,2,treatment,s19486,848,images/test/HEPG2-11/Plate3/L20_s2_w1-w6.png


In [6]:
df[
    (df.cell_type == cell_lines.hep_g2_cell)
    & (df.sirna == sirnas.s19486)
    & (df.well == wells.l20)
    & (df.plate == "3")
    & (df.site == "2")
].file_keys

9097    images/test/HEPG2-11/Plate3/L20_s2_w1-w6.png
Name: file_keys, dtype: object

## duckdb

Using duckdb to query from parquet file without loading it into memory:

In [7]:
import duckdb

In [8]:
filters = (
    f"{features.cell_type} == '{cell_lines.hep_g2_cell}' and {features.sirna} =="
    f" '{sirnas.s19486}' and {features.well} == '{wells.l20}' and "
    f"{features.plate} == '3' and {features.site} == '2'"
)

In [9]:
filters

"cell_type == 'HEPG2' and sirna == 's19486' and well == 'L20' and plate == '3' and site == '2'"

In [13]:
localpath_parquet = file.stage().as_posix()

In [14]:
duckdb.from_parquet(localpath_parquet).filter(filters)

┌──────────────────┬────────────────┬───────────┬─────────┬───┬───────────┬─────────┬──────────┬──────────────────────┐
│     site_id      │    well_id     │ cell_type │ dataset │ … │ well_type │  sirna  │ sirna_id │      file_keys       │
│     varchar      │    varchar     │  varchar  │ varchar │   │  varchar  │ varchar │ varchar  │       varchar        │
├──────────────────┼────────────────┼───────────┼─────────┼───┼───────────┼─────────┼──────────┼──────────────────────┤
│ HEPG2-11_3_L20_2 │ HEPG2-11_3_L20 │ HEPG2     │ test    │ … │ treatment │ s19486  │ 848      │ images/test/HEPG2-…  │
├──────────────────┴────────────────┴───────────┴─────────┴───┴───────────┴─────────┴──────────┴──────────────────────┤
│ 1 rows                                                                                         12 columns (8 shown) │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [15]:
duckdb.from_parquet(localpath_parquet).filter(filters).file_keys

┌──────────────────────────────────────────────┐
│                  file_keys                   │
│                   varchar                    │
├──────────────────────────────────────────────┤
│ images/test/HEPG2-11/Plate3/L20_s2_w1-w6.png │
└──────────────────────────────────────────────┘

In [5]:
# close the SQLite instance
!lamin close

2023-09-21 14:31:35,385:INFO - Found credentials in shared credentials file: ~/.aws/credentials
❗ updating & unlocking cloud SQLite 's3://lamindata/rxrx1-2.lndb' of instance 'sunnyosun/rxrx1-2'
✅ closed instance: sunnyosun/rxrx1-2
